In [6]:
from numpy import asarray
import pandas as pd
import os.path
import pickle 
import numpy as np
import keras.utils
import time
from keras.callbacks import TensorBoard, CSVLogger
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM,Conv1D,GlobalMaxPool1D,Dropout,Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.models import Model
from keras.utils.vis_utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model
from nltk.corpus import stopwords
import operator,re
import jieba.posseg as pseg

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\13051\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
# import spacy
# #!python -m spacy download en_core_web_sm
# nlp = spacy.load('en_core_web_sm')

In [8]:
import pandas as pd
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

In [9]:
train_data1 = pd.read_csv('./train.csv', encoding='UTF-8')[['multi','text','user_verified', 'user_description', 'user_gender', 'user_messages', 'user_followers', 'user_location', 'user_time', 'user_friends', 'has_url', 'comments', 'pics', 'likes', 'time', 'reposts', 'rumor','multi_type','user']]
val_data1 = pd.read_csv('./valid.csv',encoding='UTF-8')[['multi','text','user_verified', 'user_description', 'user_gender', 'user_messages', 'user_followers', 'user_location', 'user_time', 'user_friends', 'has_url', 'comments', 'pics', 'likes', 'time', 'reposts', 'rumor','multi_type','user']]
test_data1 = pd.read_csv('./test.csv', encoding='UTF-8')[['multi','text','user_verified', 'user_description', 'user_gender', 'user_messages', 'user_followers', 'user_location', 'user_time', 'user_friends', 'has_url', 'comments', 'pics', 'likes', 'time', 'reposts', 'rumor','multi_type','user']]

FileNotFoundError: [Errno 2] File b'D:/train.csv' does not exist: b'D:/train.csv'

In [6]:
train_data=train_data1[~train_data1['user'].isin(['empty'])] #删除user="empty"的行
val_data=val_data1[~val_data1['user'].isin(['empty'])]
test_data=test_data1[~test_data1['user'].isin(['empty'])] 

train_data.loc[train_data['multi'].isnull(),'multi']=train_data[train_data['multi'].isnull()]['multi_type'] #当multi为空时用multi_type替代
val_data.loc[val_data['multi'].isnull(),'multi']=val_data[val_data['multi'].isnull()]['multi_type'] 
test_data.loc[test_data['multi'].isnull(),'multi']=test_data[test_data['multi'].isnull()]['multi_type'] 



D:\pro\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
# train_data.head()

In [8]:
#####################
### OUTPUT LABELS ###
#####################

y_label_dict = {"yes" : 1, "no" : 0}
print(y_label_dict)

train_data['rumor_id'] = train_data['rumor'].apply(lambda x: y_label_dict[x])
val_data['rumor_id'] = val_data['rumor'].apply(lambda x: y_label_dict[x])
test_data['rumor_id'] = test_data['rumor'].apply(lambda x: y_label_dict[x])

num_classes = 2

{'yes': 1, 'no': 0}


D:\pro\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
########################
### USER_VERIFIED_DICT ###
########################

user_verified_dict = {"t" : 1, "f" : 0}
print(user_verified_dict)

train_data['user_verified_id'] = train_data['user_verified'].apply(lambda x: user_verified_dict[x])
val_data['user_verified_id'] = val_data['user_verified'].apply(lambda x: user_verified_dict[x])
test_data['user_verified_id'] = test_data['user_verified'].apply(lambda x: user_verified_dict[x])

num_classes = 2

{'t': 1, 'f': 0}


D:\pro\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
###########################
### USER_DISCRIPTION_DICT ###
###########################

user_description_dict = {"t" : 1, "f" : 0}
print(user_description_dict)

train_data['user_description_id'] = train_data['user_description'].apply(lambda x: user_description_dict[x])
val_data['user_description_id'] = val_data['user_description'].apply(lambda x: user_description_dict[x])
test_data['user_description_id'] = test_data['user_description'].apply(lambda x: user_description_dict[x])

num_classes = 2

{'t': 1, 'f': 0}


D:\pro\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
###########################
### USER_GENDER_DICT ###
###########################

user_gender_dict = {"f" : 1, "m" : 0}
print(user_gender_dict)

train_data['user_gender_id'] = train_data['user_gender'].apply(lambda x: user_gender_dict[x])
val_data['user_gender_id'] = val_data['user_gender'].apply(lambda x: user_gender_dict[x])
test_data['user_gender_id'] = test_data['user_gender'].apply(lambda x: user_gender_dict[x])

num_classes = 2

{'f': 1, 'm': 0}


D:\pro\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
#########################
### USER_MESSAGE_DICT ###
#########################
print(train_data['user_messages'].mode())
bins=[0,]
# 1947 max=335237  min=0 median=11552  mean=28219.43
# user_message_dict = {"f" : 1, "m" : 0}
# print(user_gender_dict)

# train_data['user_gender_id'] = train_data['user_gender'].apply(lambda x: user_gender_dict[x])
# val_data['user_gender_id'] = val_data['user_gender'].apply(lambda x: user_gender_dict[x])
# test_data['user_gender_id'] = test_data['user_gender'].apply(lambda x: user_gender_dict[x])

# num_classes = 2

0    118036.0
dtype: float64


In [13]:
###########################
### HAS_URL_DICT ###
###########################

has_url_dict = {"t" : 1, "f" : 0}
print(has_url_dict)

train_data['has_url_id'] = train_data['has_url'].apply(lambda x: has_url_dict[x])
val_data['has_url_id'] = val_data['has_url'].apply(lambda x: has_url_dict[x])
test_data['has_url_id'] = test_data['has_url'].apply(lambda x: has_url_dict[x])

num_classes = 2

{'t': 1, 'f': 0}


D:\pro\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
########################
###   USER_LOCATION    ###
########################

#frequent_speakers = train_data['speaker'].value_counts()[:20].reset_index().to_dict()['index']
#frequent_speakers = dict((v,k) for k,v in frequent_speakers.iteritems())

frequent_location = {'安徽' : 0, '澳门' : 1, '北京' : 2, '福建' : 3, '甘肃' : 4, '广东' : 5, 
                     '广西' : 6, '贵州' : 7, '海南': 8, '海外':9, '河北':10, '河南':11, '黑龙':12, 
                     '湖北' : 13, '湖南': 14, '吉林' : 15, '江苏':16,'江西':17, '辽宁':18,'内蒙' : 19, 
                     '其他' : 20,'山东':21, '山西' : 22, '陕西' : 23, '上海' : 24, '四川':25,'台湾':26, 
                     '天津':27, '西藏':28, '香港':29, '新疆':30, '云南':31, '浙江':32, '重庆':33, 
                     '青海':34, '宁夏':35}
#print(frequent_location)

key_ls=frequent_location.keys()
def get_location_id(location):
    if isinstance(location, str):
        if location[:2] in key_ls:
            return frequent_location[location[:2]]
        else:
            return len(set(frequent_location.values())) 
    else:
        return len(set(frequent_location.values())) 

# key_ls=frequent_location.keys()
# def get_location_id(location):
#     if location[:2] in key_ls:
#         return len(frequent_location.values())

train_data['location_id'] = train_data['user_location'].apply(get_location_id)
val_data['location_id'] = val_data['user_location'].apply(get_location_id)
test_data['location_id'] = test_data['user_location'].apply(get_location_id)

print(len(set(frequent_location.values())))

train_data['location_id'].value_counts()

36


D:\pro\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2     628
5     320
24    194
9     154
20    86 
32    67 
16    62 
29    48 
25    46 
3     40 
14    34 
26    33 
21    32 
13    28 
33    19 
11    18 
0     17 
18    15 
23    14 
27    12 
10    9  
6     9  
17    8  
8     8  
22    7  
12    7  
31    7  
7     5  
4     5  
30    5  
15    4  
19    2  
28    2  
1     2  
Name: location_id, dtype: int64

In [15]:
##############
###   TEXT   ###
##############

stopwords = [line.strip() for line in open('D:/stopwordS.txt', 'r', encoding='utf-8').readlines()]

def proc_text(text):
    filter_pattern = re.compile('[^\u4E00-\u9FA5]+')
    chinese_only = filter_pattern.sub('', text)
    words_lst = pseg.cut(chinese_only)
    meaninful_words = []
    for word, flag in words_lst:
#      if (word not in stopwords) and (flag == 'v'):
            # 也可根据词性去除非动词等
    # handle stopwords:
        if word not in stopwords :   #len(word) >= 2 and 
            meaninful_words.append(word)
    return ' '.join(meaninful_words)

def load_statement_vocab_dict(train_data):
    vocabulary_dict = {}
    if not os.path.exists('vocabulary.p'):
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(train_data)
        vocabulary_dict = tokenizer.word_index
        print(len(vocabulary_dict))
        pickle.dump(vocabulary_dict, open( "vocabulary.p", "wb" ))
        print('Created Vocabulary Dictionary...')
        print('Saved Vocabulary Dictionary...')
    else:
        print('Loading Vocabulary Dictionary...')
        vocabulary_dict = pickle.load(open("vocabulary.p", "rb" ))
    return vocabulary_dict


def preprocess_statement(text):
    val = [0] * 10
    val = [vocabulary_dict[t] for t in text if t in vocabulary_dict] 
    return val


train_pre = train_data['text'].apply(proc_text)
val_pre = val_data['text'].apply(proc_text)
test_pre = test_data['text'].apply(proc_text)

vocabulary_dict = load_statement_vocab_dict(train_pre)

train_data['word_id'] = train_pre.apply(preprocess_statement)
val_data['word_id'] = val_pre.apply(preprocess_statement)
test_data['word_id'] = test_pre.apply(preprocess_statement)

# 过滤空字符串
# train_data = train_data[train_data['text'] != '']
# val_data = val_data[val_data['text'] != '']
# test_data = test_data[test_data['text'] != '']

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\180773~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.673 seconds.
Prefix dict has been built succesfully.


18124
Created Vocabulary Dictionary...
Saved Vocabulary Dictionary...


D:\pro\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
#####################
###  EMBEDDINGS   ###
#####################

embeddings = {}
with open("D:/sgns.weibo.txt", encoding='utf-8', errors='ignore') as file_object:
  for line in file_object:
    word_embed = line.split()
    word = word_embed[0]
    embed = np.array(word_embed[1:])
    embeddings[word.lower()]= embed

EMBED_DIM = 300
print(len(embeddings), " : Word Embeddings Found")
print(len(embeddings[word]), " : Embedding Dimension")


num_words = len(vocabulary_dict) + 1
embedding_matrix = np.zeros((num_words, EMBED_DIM))
for word, i in vocabulary_dict.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embeddings_index = None

189595  : Word Embeddings Found
300  : Embedding Dimension


In [17]:
embedding_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.912511,  0.15435 ,  0.519588, ..., -0.536558, -0.357079,
         0.75555 ],
       [ 0.65388 ,  0.756262,  0.659758, ...,  0.008703, -0.100621,
         0.610115],
       ...,
       [-0.282047, -0.011514,  0.681716, ..., -0.113365,  0.486522,
        -0.553472],
       [ 0.915986,  0.242119,  0.061393, ..., -0.32598 ,  0.160503,
        -0.034218],
       [ 0.380722, -0.26947 ,  0.631396, ..., -1.495936, -0.052521,
        -0.033431]])

In [18]:
#####################
###  HYPERPARAMS  ###
#####################
vocab_length = len(vocabulary_dict.keys())
hidden_size = EMBED_DIM #Has to be same as EMBED_DIM
lstm_size = 100
num_steps = 35
num_epochs = 30
batch_size = 64     #批量大小，决定一次训练的样本数目，batchsize 的正确选择是为了在内存效率和内存容量之间寻找最佳平衡
#batch_size为一个batch中样本的数量（当成一个batch的行），num_steps是一个样本的序列长度（当成一个batch的列）
#num_steps 表示一次有15个单词输入给相连的15个lstm神经元
#batch_size 表示一个迭代里有40组这样的15个单词的样本
#one epoch = numbers of iterations = N = 训练样本的数量/batch_size

#Hyperparams for CNN
kernel_sizes = [3,3,3]
filter_size = 128

#Meta data related hyper params
num_verified = len(train_data.user_verified_id.unique())
num_description = len(train_data.user_description_id.unique())
num_gender = len(train_data.user_gender_id.unique())
num_location = len(train_data.location_id.unique())+2
num_has_url = len(train_data.has_url_id.unique())

print(num_verified)
print(num_description)
print(num_gender)
print(num_location)
print(num_has_url)

print(train_data.columns)

2
2
2
36
2
Index(['multi', 'text', 'user_verified', 'user_description', 'user_gender',
       'user_messages', 'user_followers', 'user_location', 'user_time',
       'user_friends', 'has_url', 'comments', 'pics', 'likes', 'time',
       'reposts', 'rumor', 'multi_type', 'user', 'rumor_id',
       'user_verified_id', 'user_description_id', 'user_gender_id',
       'has_url_id', 'location_id', 'word_id'],
      dtype='object')


In [19]:
#####################
###   DATA PREP   ###
#####################

# word_id - statement
X_train = train_data['word_id']
X_val = val_data['word_id']
X_test = test_data['word_id']

# ouput - label
Y_train = train_data['rumor_id']
Y_train = keras.utils.to_categorical(Y_train, num_classes=2)

Y_val = val_data['rumor_id']
Y_val = keras.utils.to_categorical(Y_val, num_classes=2)

X_train = sequence.pad_sequences(X_train, maxlen=num_steps, padding='post',truncating='post')
X_val = sequence.pad_sequences(X_val, maxlen=num_steps, padding='post',truncating='post')
X_test = sequence.pad_sequences(X_test, maxlen=num_steps, padding='post',truncating='post')


#Meta data preparation
verified_train = keras.utils.to_categorical(train_data['user_verified_id'], num_classes=num_verified)
description_train = keras.utils.to_categorical(train_data['user_description_id'], num_classes=num_description)
gender_train = keras.utils.to_categorical(train_data['user_gender_id'], num_classes=num_gender)
location_train = keras.utils.to_categorical(train_data['location_id'], num_classes=num_location)
has_url_train = keras.utils.to_categorical(train_data['has_url_id'], num_classes=num_has_url)

# X_train_meta = location_train
X_train_meta = np.hstack((verified_train, description_train, gender_train, location_train, has_url_train))

verified_val = keras.utils.to_categorical(val_data['user_verified_id'], num_classes=num_verified)
description_val = keras.utils.to_categorical(val_data['user_description_id'], num_classes=num_description)
gender_val = keras.utils.to_categorical(val_data['user_gender_id'], num_classes=num_gender)
location_val = keras.utils.to_categorical(val_data['location_id'], num_classes=num_location)
has_url_val = keras.utils.to_categorical(val_data['has_url_id'], num_classes=num_has_url)

#X_val_meta = location_val
X_val_meta = np.hstack((verified_val, description_val, gender_val, location_val, has_url_val))

verified_test = keras.utils.to_categorical(test_data['user_verified_id'], num_classes=num_verified)
description_test = keras.utils.to_categorical(test_data['user_description_id'], num_classes=num_description)
gender_test = keras.utils.to_categorical(test_data['user_gender_id'], num_classes=num_gender)
location_test = keras.utils.to_categorical(test_data['location_id'], num_classes=num_location)
has_url_test = keras.utils.to_categorical(test_data['has_url_id'], num_classes=num_has_url)

#X_test_meta = location_test
X_test_meta = np.hstack((verified_test, description_test, gender_test, location_test, has_url_test))

In [20]:
print(X_train_meta.shape, X_val_meta.shape, X_test_meta.shape)
print(X_train.shape, X_val.shape, X_test.shape)
print(Y_train.shape, Y_val.shape)

(1947, 44) (678, 44) (674, 44)
(1947, 35) (678, 35) (674, 35)
(1947, 2) (678, 2)


In [21]:
def train(model, name, use_pos = False, use_meta = False, use_dep = False):
    sgd = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    adam = optimizers.Adam(lr=0.025, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    # optimizer制定优化方式，loss制定损失函数，metrics指定衡量模型的指标
    # “categorical_crossentropy”多分类的对数损失函数，与softmax分类器相对应的损失函数
    #　tip：此损失函数与上一类同属对数损失函数，sigmoid和softmax的区别主要是，sigmoid用于二分类，softmax用于多分类
    tb = TensorBoard()
    csv_logger = keras.callbacks.CSVLogger('training.log')
    filepath= name+"_weights_best.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy', 
                                             verbose=1, save_best_only=True, mode='max')
  
  
    model.fit(
        {'main_input': X_train},
        {'main_output': Y_train}, epochs = num_epochs, batch_size = batch_size,
        validation_data = (
            {'main_input': X_val},
            {'main_output': Y_val}
        ), callbacks=[tb,csv_logger,checkpoint])
  

In [22]:
kernel_stmt = []
kernel_pos = []
kernel_dep = []

use_pos = False
use_meta = False
use_dep = False

statement_input = Input(shape=(num_steps,), dtype='int32', name='main_input')
x_stmt = Embedding(vocab_length+1,EMBED_DIM,weights=[embedding_matrix],input_length=num_steps,trainable=False)(statement_input) 

for kernel in kernel_sizes:
    x_1 = Conv1D(filters=filter_size,kernel_size=kernel)(x_stmt)
    x_1 = GlobalMaxPool1D()(x_1)
    kernel_stmt.append(x_1)
    

    
conv_in1 = keras.layers.concatenate(kernel_stmt)
conv_in1 = Dropout(0.7)(conv_in1)
conv_in1 = Dense(128, activation='relu')(conv_in1)



#meta data
meta_input = Input(shape=(X_train_meta.shape[1],), name='aux_input')
x_meta = Dense(64, activation='relu')(meta_input)

# x = keras.layers.concatenate([conv_in1, x_meta])    #text+meta
x = conv_in1     #text

main_output = Dense(2, activation='softmax', name='main_output')(x)
# model_cnn = Model(inputs=[statement_input, meta_input], outputs=[main_output])     #text+meta
model_cnn = Model(inputs=[statement_input], outputs=[main_output])   #text


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [23]:
# print(model_cnn.summary())

In [24]:
train(model_cnn,'cnn')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1947 samples, validate on 678 samples
Epoch 1/30
1947/1947 [==============================] - 2s 1ms/step - loss: 0.9649 - categorical_accuracy: 0.5177 - val_loss: 0.6525 - val_categorical_accuracy: 0.5885

Epoch 00001: val_categorical_accuracy improved from -inf to 0.58850, saving model to cnn_weights_best.hdf5
Epoch 2/30
1947/1947 [==============================] - 1s 762us/step - loss: 0.6821 - categorical_accuracy: 0.6009 - val_loss: 0.6233 - val_categorical_accuracy: 0.6136

Epoch 00002: val_categorical_accuracy improved from 0.58850 to 0.61357, saving model to cnn_weights_best.hdf5
Epoch 3/30
1947/1947 [==============================] - 1s 742us/step - loss: 0.5897 - categorical_accuracy: 0.6893 - val_loss: 0.5206 - val_categorical_accuracy: 0.7773

Epoch 00003: val_categorical_accuracy improved from 0.61357 to 0.77729, saving model to cnn_weights

In [25]:
def evaluate(name, use_meta=False):
    model1 = load_model(name+'_weights_best.hdf5')
    if use_pos and use_meta:
        if use_dep:
            preds = model1.predict([X_test,X_test_pos, X_test_dep, X_test_meta], batch_size=batch_size, verbose=1)
        else:
            preds = model1.predict([X_test,X_test_pos, X_test_meta], batch_size=batch_size, verbose=1)
    elif use_meta:
        if use_dep:
            preds = model1.predict([X_test, X_test_dep, X_test_meta], batch_size=batch_size, verbose=1)
        else:
            preds = model1.predict([X_test, X_test_meta], batch_size=batch_size, verbose=1)
    elif use_pos:
        if use_dep:
            preds = model1.predict([X_test, X_test_pos, X_test_dep], batch_size=batch_size, verbose=1)
        else:
            preds = model1.predict([X_test, X_test_pos], batch_size=batch_size, verbose=1)
    else:
        if use_dep:
            preds = model1.predict([X_test, X_test_dep], batch_size=batch_size, verbose=1)
        else:
            preds = model1.predict([X_test], batch_size=batch_size, verbose=1)
    false_worst = {}
    true_best = {}
    label_list = ['yes','no']
    
    Y_test_gt = list(test_data['rumor_id'])
    predictions = np.array([np.argmax(pred) for pred in preds])
    
    print(len(predictions)==len(Y_test_gt))
    correct = np.sum(predictions == Y_test_gt)
    acc = correct*100.0/len(Y_test_gt)
    print("Correctly Predicted")
    print("Accuracy : ")
    pickle.dump(predictions, open(name+'_predictions.p','wb'))
    return acc

In [26]:
evaluate('cnn')

674/674 [==============================] - 0s 372us/step
True
Correctly Predicted
Accuracy : 


86.64688427299703